# Community Finding - Algorithm Comparision
## Jake Carlson

## Load Data

In [1]:
import numpy as np
import pandas as pd
import igraph
import uuid
import matplotlib.pyplot as plt

df_q = pd.read_csv("./data/2008-questions.csv")
df_a = pd.read_csv("./data/2008-answers.csv")
df_t = pd.read_csv("./data/2008-tags.csv")

# remove NaN owner ids
df_q = df_q[np.isfinite(df_q.OwnerUserId.values)]
df_a = df_a[np.isfinite(df_a.OwnerUserId.values) & df_a.ParentId.isin(df_q.Id)]

df_q.OwnerUserId = df_q.OwnerUserId.astype(np.int)
df_a.OwnerUserId = df_a.OwnerUserId.astype(np.int)

df_q['qid'] = [str(uuid.uuid4()) for _ in range(len(df_q))]
df_a['qid'] = [df_q['qid'].values[df_q.Id == a][0] for a in df_a.ParentId.values]

In [2]:
owner_to_uid = {}
def get_uids(df):
    uids = []
    for i, r in df.iterrows():
        owner = r['OwnerUserId']
        uid = None
        if owner in owner_to_uid.keys():
            uid = owner_to_uid[owner]
        else:
            uid = str(uuid.uuid4())
            owner_to_uid[owner] = uid
        uids.append(uid)
    return uids

df_q['uid'] = get_uids(df_q)
df_a['uid'] = get_uids(df_a)

In [3]:
df_q.head()

,Id,OwnerUserId,CreationDate,Score,Title,qid,uid
0,80,26,2008-08-01T13:57:07Z,26,SQLStatement.execute() - multiple queries in o...,3047e7dd-e0df-49c6-89d5-361fe91d180c,e6af2e8d-a900-4733-90b8-27406c5f3048
1,90,58,2008-08-01T14:41:24Z,144,Good branching and merging tutorials for Torto...,04c7710e-9f2a-46f5-8e44-034f29ee548e,6224eac1-0305-41b1-923c-7e6ecbc48b31
2,120,83,2008-08-01T15:50:08Z,21,ASP.NET Site Maps,96bd567c-d7f0-4cfb-acf7-f6fd20dd846c,5c647ab6-4424-469c-8e2e-33fec25e6311
3,180,2089740,2008-08-01T18:42:19Z,53,Function for creating color wheels,0c6cabb1-cc72-4fcb-bda1-d21f9f553340,61a903ee-661e-482d-93b6-4cf5acdeafdf
4,260,91,2008-08-01T23:22:08Z,49,Adding scripting functionality to .NET applica...,3d6ba28b-ab22-4d7d-85db-e5ae525be249,4341f7a4-26a7-493e-92ea-31346c33fb77


In [4]:
df_a.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,qid,uid
0,92,61,2008-08-01T14:45:37Z,90,13,04c7710e-9f2a-46f5-8e44-034f29ee548e,01d24177-d01f-4d53-9017-2589de4584fb
1,124,26,2008-08-01T16:09:47Z,80,12,3047e7dd-e0df-49c6-89d5-361fe91d180c,e6af2e8d-a900-4733-90b8-27406c5f3048
2,199,50,2008-08-01T19:36:46Z,180,1,0c6cabb1-cc72-4fcb-bda1-d21f9f553340,7c8180a0-f62a-4391-aae3-4b72c2807b4a
3,269,91,2008-08-01T23:49:57Z,260,4,3d6ba28b-ab22-4d7d-85db-e5ae525be249,4341f7a4-26a7-493e-92ea-31346c33fb77
4,307,49,2008-08-02T01:49:46Z,260,28,3d6ba28b-ab22-4d7d-85db-e5ae525be249,d1bf1833-da23-402c-bb71-f5ddd0eac248


In [5]:
df_t.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [6]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16912 entries, 0 to 16911
Data columns (total 2 columns):
Id     16912 non-null int64
Tag    16912 non-null object
dtypes: int64(1), object(1)
memory usage: 264.3+ KB


In [7]:
df_q.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5299 entries, 0 to 5824
Data columns (total 7 columns):
Id              5299 non-null int64
OwnerUserId     5299 non-null int64
CreationDate    5299 non-null object
Score           5299 non-null int64
Title           5299 non-null object
qid             5299 non-null object
uid             5299 non-null object
dtypes: int64(3), object(4)
memory usage: 331.2+ KB


## Questions Only

In [8]:
tags_to_questions = {}
for g, data in df_t.groupby(['Tag']):
    tags_to_questions[g] = set(df_q.qid[df_q.Id.isin(data.Id)].values)

questions_to_questions = {}
for t, quids in tags_to_questions.items():
    for i, q in enumerate(quids):
        if q in questions_to_questions.keys():
            questions_to_questions[q].update(quids)
        else:
            questions_to_questions[q] = set(quids)
        questions_to_questions[q].remove(q)

questions = list(questions_to_questions.keys())
questions_to_idx = {}
for i, q in enumerate(questions):
    questions_to_idx[q] = i

In [9]:
edges = []
for k, v in questions_to_questions.items():
    for q in v:
        edges.append((questions_to_idx[k], questions_to_idx[q]))

In [10]:
graph = igraph.Graph(edges=edges, directed=False).simplify(multiple=True, loops=False)
graph.vs['name'] = questions

In [11]:
def get_largest_component(g):
    comps = g.components()
    sizes = comps.sizes()
    idx_largest = sizes.index(max(sizes))
    return comps.subgraph(idx_largest)

In [12]:
largest = get_largest_component(graph)

## Spectral Bisection

In [13]:
def get_smallest_component(g):
    comps = g.components()
    sizes = comps.sizes()
    idx_largest = sizes.index(2)
    return comps.subgraph(idx_largest)

smallest = get_smallest_component(graph)

In [52]:
import importlib
import methods

importlib.reload(methods)

methods.spectral_bisection(largest)
# methods.spectral_bisection(smallest)

set()
set()


## Walktrap

In [ ]:
dendrogram = largest.community_walktrap()
dendrogram.optimal_count

In [ ]:
tag_counts = df_t.Tag.value_counts()
tag_counts[:100].plot()
plt.show()

In [ ]:
clusters = dendrogram.as_clustering()
membership = clusters.membership
largest.vs['membership'] = membership

In [ ]:
def get_n_tag_counts_for_membership(m, n):
    q_ids = set([graph.vs[i]['name'] for i in range(len(largest.vs)) if membership[i] == m])
    q_tags = df_t.Tag[df_t.Id.isin(df_q.Id[df_q.qid.isin(q_ids)])]
    return len(q_ids), q_tags.value_counts()[:n]

In [ ]:
for m in list(set(membership)):
    num, counts = get_n_tag_counts_for_membership(m, 5)
    print(m, ":", num)
    print(counts)

In [ ]:
def show_n_questions_for_membership(m, n):
    q_ids = set([graph.vs[i]['name'] for i in range(len(largest.vs)) if membership[i] == m])
    q_titles = df_q.Title.values[df_q.qid.isin(q_ids)]
    return q_titles[:n]

In [ ]:
show_n_questions_for_membership(2, 10)

In [ ]:
def plot_memberships(ms):
    palette = igraph.drawing.colors.ClusterColoringPalette(len(clusters))
    colors = palette.get_many(membership)
    largest.vs['color'] = colors
    layout = largest.layout_drl()
    p = igraph.drawing.Plot(background='white')
    for m in ms:
        p.add(clusters.subgraph(m), opacity=0.6, layout=layout)
    return p

In [ ]:
plot = plot_memberships([0,1])
plot.show()

In [ ]:
palette = igraph.drawing.colors.ClusterColoringPalette(len(clusters))
colors = palette.get_many(membership)

# set user colors (r, b, g, a)
# for i in range(len(users)):
#     colors[i] = (1.0, 1.0, 1.0, 1.0)
# for i in range(len(questions)):
#     colors[offset+i] = (1.0, 0.0, 0.0, 1.0)
largest.vs['color'] = colors
igraph.plot(largest, opacity=0.5)

## Spinglass

In [ ]:
clustering = largest.community_spinglass()
membership = clustering.membership

In [ ]:
for m in list(set(membership)):
    num, counts = get_n_tag_counts_for_membership(m, 5)
    print(m, ":", num)
    print(counts)

In [ ]:
palette = igraph.drawing.colors.ClusterColoringPalette(len(set(membership)))
colors = palette.get_many(membership)

# set user colors (r, b, g, a)
# for i in range(len(users)):
#     colors[i] = (1.0, 1.0, 1.0, 1.0)
# for i in range(len(questions)):
#     colors[offset+i] = (1.0, 0.0, 0.0, 1.0)
largest.vs['color'] = colors
igraph.plot(largest, opacity=0.5)